Import all variables/ libraries from main

In [1]:
from main import data, pd, plt, train_test_split, GaussianNB, accuracy_score, DecisionTreeClassifier, export_text, plot_tree, sns, apriori, association_rules, data_x, target
import constants
import utils
import pprint

pp = pprint.PrettyPrinter(indent=4)

Print data

In [2]:
data

,Systemic_Illness,Rectal_Pain,Sore_Throat,Penile_Oedema,Oral_Lesions,Solitary_Lesion,Swollen_Tonsils,HIV_Infection,STI,MonkeyPox
Patient_ID,,,,,,,,,,
P0,None,False,True,True,True,False,True,False,False,Negative
P1,Fever,True,False,True,True,False,False,True,False,Positive
P2,Fever,False,True,True,False,False,False,True,False,Positive
P3,None,True,False,False,False,True,True,True,False,Positive
P4,Swollen Lymph Nodes,True,True,True,False,False,True,True,False,Positive
...,...,...,...,...,...,...,...,...,...,...
P24995,None,True,True,False,True,True,False,False,True,Positive
P24996,Fever,False,True,True,False,True,True,True,True,Positive
P24997,None,True,True,False,False,True,True,False,False,Positive


Plot data, shows how often MonkeyPox is negative and positive

In [3]:
series = data[constants.MONKEY_POX].value_counts()
#series2,
# TODO: Try to use countplot and change colors to normal blue and orange
plt.bar(series.index, series.values, color=constants.COLOR_PALETTE)
plt.title('Cases of Monkey Pox seperated by positive and negative')
plt.show()

In [4]:
df_corr  = data.corr()
df_corr

/tmp/ipykernel_25966/1271232218.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_corr  = data.corr()


,Rectal_Pain,Sore_Throat,Penile_Oedema,Oral_Lesions,Solitary_Lesion,Swollen_Tonsils,HIV_Infection,STI
Rectal_Pain,1.000000,0.006214,0.005312,0.007747,0.001147,0.002113,0.005764,0.000666
Sore_Throat,0.006214,1.000000,0.009082,-0.000795,-0.003609,-0.002011,0.000771,-0.001901
Penile_Oedema,0.005312,0.009082,1.000000,0.003850,0.002781,0.000536,0.003620,-0.005722
Oral_Lesions,0.007747,-0.000795,0.003850,1.000000,-0.005198,-0.007757,-0.012153,-0.002725
Solitary_Lesion,0.001147,-0.003609,0.002781,-0.005198,1.000000,0.002874,-0.002335,0.009849
Swollen_Tonsils,0.002113,-0.002011,0.000536,-0.007757,0.002874,1.000000,-0.004258,-0.006469
HIV_Infection,0.005764,0.000771,0.003620,-0.012153,-0.002335,-0.004258,1.000000,0.001309
STI,0.000666,-0.001901,-0.005722,-0.002725,0.009849,-0.006469,0.001309,1.000000


Plots heatmap: Shows correlations between the different features

In [5]:
sns.heatmap(df_corr, annot=True, vmin=0.5, vmax=0.7)
plt.show()

Shows how often each systemic illness is causing MonkeyPox to be positive or negative

In [6]:
# lift = 1
# confidence = 1 is good?
symptoms_df = utils.clean_data(data)
frequency = apriori(symptoms_df, min_support=0.005, use_colnames=True)
rules = association_rules(frequency, metric='lift')
rules.sort_values('confidence', ascending=False, inplace=True)
rules

/home/linus/Daten_Linus/Studium/IW7/Data_Mining/Project_Data_Mining/utils.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat[col][index] = True


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
20252,"(Penile_Oedema, None, STI, Swollen_Tonsils, Or...",(Rectal_Pain),0.015086,0.546986,0.009114,0.604167,1.104538,0.000863,1.144456
36020,"(Swollen Lymph Nodes, STI, Rectal_Pain, Swolle...",(Sore_Throat),0.010246,0.526117,0.006160,0.601227,1.142762,0.000770,1.188352
19694,"(None, STI, Sore_Throat, HIV_Infection, Oral_L...",(Rectal_Pain),0.014646,0.546986,0.008737,0.596567,1.090643,0.000726,1.122896
31111,"(Penile_Oedema, Fever, STI, Sore_Throat, Recta...",(Solitary_Lesion),0.010874,0.515243,0.006474,0.595376,1.155524,0.000871,1.198042
20005,"(Penile_Oedema, None, Solitary_Lesion, Swollen...",(Rectal_Pain),0.013452,0.546986,0.007983,0.593458,1.084960,0.000625,1.114310
...,...,...,...,...,...,...,...,...,...
36640,(Rectal_Pain),"(Penile_Oedema, Swollen Lymph Nodes, STI, Swol...",0.546986,0.011189,0.005029,0.009193,0.821663,-0.001091,0.997986
37776,(HIV_Infection),"(Penile_Oedema, STI, Sore_Throat, Rectal_Pain,...",0.558615,0.009303,0.005091,0.009114,0.979740,-0.000105,0.999810
36642,(HIV_Infection),"(Penile_Oedema, Swollen Lymph Nodes, STI, Rect...",0.558615,0.010120,0.005029,0.009002,0.889512,-0.000625,0.998872
37903,(HIV_Infection),"(Penile_Oedema, STI, Sore_Throat, Rectal_Pain,...",0.558615,0.010057,0.005029,0.009002,0.895071,-0.000589,0.998935


In [7]:
# TODO: Change legend and axises
sns.countplot(x=data[constants.SYSTEMIC_ILLNESS], hue=data[constants.MONKEY_POX], palette=constants.COLOR_PALETTE)
plt.show()

Plots data, using pairplot

In [ ]:
#sns.pairplot(data.head(100), hue='MonkeyPox')

Calculates accuracy, using GaussianNB()

In [8]:
ran_stream = 23
x_train, x_test, y_train, y_test = train_test_split(data_x, target, random_state=ran_stream)

model = GaussianNB()
model.fit(x_train, y_train)
y_prediction = model.predict(x_test)
print(100 * accuracy_score(y_test, y_prediction))

67.984


Creates DecisionTree, by using entropy

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data_x, target, test_size=0.30)
model_tree = DecisionTreeClassifier(criterion='entropy', splitter='best', min_samples_split=0.10)
model_tree.fit(x_train, y_train)
# Accuracy checken.
y_prediction = model_tree.predict(x_test)
#print(100 * accuracy_score(y_test, y_prediction))
text_representation = export_text(model_tree, feature_names=x_train.columns.values.tolist())
figure = plt.figure(figsize=(10,8))
plot_tree(model_tree, feature_names=x_train.columns, filled=True, rounded=True)
plt.show()

Plots data, when symptom (2-7) is True and MonkeyPox is Positive

In [10]:
symptoms = data.iloc[:, 1:-1].columns.values.tolist()
symptoms_df = data.iloc[:, 1:]
symptoms_df = utils.fetch_true_symptoms(symptoms, symptoms_df)
print(symptoms_df)

pos_lst, negative_lst = utils.create_outcome_lists(symptoms, symptoms_df)
df1 = pd.DataFrame({'Symptoms': symptoms, 'Number of infections': pos_lst})
df2 = pd.DataFrame({'Symptoms': symptoms, 'Number of infections': negative_lst})
df1['outcome'] = 'Positive'
df2['outcome'] = 'Negative'
res = pd.concat([df1, df2])
sns.barplot(x='Symptoms', y='Number of infections', data=res, hue='outcome', palette=constants.COLOR_PALETTE)
plt.show()

          Rectal_Pain  Sore_Throat  Penile_Oedema  Oral_Lesions  \
Negative         3643         4184           4214          4228   
Positive         8702         8370           8398          8258   

          Solitary_Lesion  Swollen_Tonsils  HIV_Infection   STI  
Negative             4330             4478           3697  3784  
Positive             8197             8055           8887  8662  


Calculates accuracy by removing different symptoms, checks if accuracy is higher or lower depending on which symptom is removed

In [11]:
temp_dict = utils.create_feature_accuracy_dict(data_x, target)
top_items = sorted(temp_dict.items(), key = lambda x: x[1], reverse=True)[:5]
pp.pprint(top_items)

[   (   (   "DecisionTreeClassifier(criterion='entropy', min_samples_split=5)",
            "['Systemic_Illness', 'Rectal_Pain', 'Sore_Throat', "
            "'HIV_Infection', 'STI']"),
        68.8),
    (   (   'RandomForestClassifier()',
            "['Systemic_Illness', 'Rectal_Pain', 'Sore_Throat', "
            "'HIV_Infection', 'STI']"),
        68.8),
    (   (   "DecisionTreeClassifier(criterion='entropy', min_samples_split=5)",
            "['Systemic_Illness', 'Rectal_Pain', 'Sore_Throat', "
            "'Solitary_Lesion', 'HIV_Infection', 'STI']"),
        68.768),
    (   (   "DecisionTreeClassifier(criterion='entropy', min_samples_split=5)",
            "['Systemic_Illness', 'Rectal_Pain', 'Penile_Oedema', "
            "'Solitary_Lesion', 'HIV_Infection', 'STI']"),
        68.768),
    (   (   'RandomForestClassifier()',
            "['Systemic_Illness', 'Rectal_Pain', 'Penile_Oedema', "
            "'Solitary_Lesion', 'HIV_Infection', 'STI']"),
        68.768)]


Tries to detect overfitting by plotting the test and train data

In [13]:
utils.check_over_fitting(data_x, target)

(25000, 9) (25000, 1)
